<a href="https://colab.research.google.com/github/PeterPirog/tf-autoencoders/blob/main/01_mnist_VAE_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://keras.io/examples/generative/vae/
Latent vector length =3

In [1]:
# Setup
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

epochs=50
latent_dim = 2

In [2]:
# Create a sampling layer

class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [3]:
# Build the encoder



encoder_inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()


Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 32)   320         ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 7, 7, 64)     18496       ['conv2d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 3136)         0           ['conv2d_1[0][0]']               
                                                                                            

In [4]:
# Build the decoder

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(7 * 7 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()



Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_1 (Dense)             (None, 3136)              9408      
                                                                 
 reshape (Reshape)           (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 14, 14, 64)       36928     
 nspose)                                                         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 conv2d_transpose_2 (Conv2DT  (None, 28, 28, 1)        289 

In [5]:
# Define the VAE as a Model with a custom train_step

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }



In [6]:
callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                factor=0.5,
                                                patience=3,
                                                min_lr=1e-5),
           tf.keras.callbacks.EarlyStopping(monitor='vval_loss',
                                            patience=5),
           tf.keras.callbacks.ModelCheckpoint(filepath='my_model.h5',
                                              monitor='val_loss',
                                              save_best_only=True)]

In [ ]:
# Train the VAE

(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(mnist_digits, epochs=epochs, batch_size=128)



Epoch 1/50
547/547 [==============================] - 22s 31ms/step - loss: 251.6771 - reconstruction_loss: 202.6759 - kl_loss: 4.1954
Epoch 2/50
547/547 [==============================] - 11s 20ms/step - loss: 173.8584 - reconstruction_loss: 164.9052 - kl_loss: 5.6859
Epoch 3/50
547/547 [==============================] - 10s 19ms/step - loss: 164.1541 - reconstruction_loss: 156.4130 - kl_loss: 6.0381
Epoch 4/50
547/547 [==============================] - 10s 19ms/step - loss: 159.8223 - reconstruction_loss: 152.7056 - kl_loss: 6.2007
Epoch 5/50
547/547 [==============================] - 10s 19ms/step - loss: 157.5971 - reconstruction_loss: 150.7089 - kl_loss: 6.2962
Epoch 6/50
547/547 [==============================] - 10s 19ms/step - loss: 155.5703 - reconstruction_loss: 149.1788 - kl_loss: 6.3294
Epoch 7/50
547/547 [==============================] - 10s 19ms/step - loss: 154.6055 - reconstruction_loss: 148.0601 - kl_loss: 6.3687
Epoch 8/50
547/547 [==============================] - 1

L=2
Epoch 50/50
547/547 [==============================] - 13s 24ms/step - loss: 146.2070 - reconstruction_loss: 139.6618 - kl_loss: 6.5275

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#Train data
x_train = np.expand_dims(x_train, -1).astype("float32") / 255
z_mean_train, _, _ = vae.encoder.predict(x_train)

#Test data
x_test = np.expand_dims(x_test, -1).astype("float32") / 255
z_mean_test, _, _ = vae.encoder.predict(x_test)

z_mean_train.shape



In [ ]:
model=keras.models.Sequential()
model.add(keras.Input(shape=z_mean_train.shape[1:]))
model.add(keras.layers.Dense(30,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=keras.optimizers.Adam(),
              metrics=[keras.metrics.SparseCategoricalAccuracy()]) # 

In [ ]:
model.fit(x=z_mean_train,y=y_train,
          epochs=30,
          batch_size=32,
          validation_data=(z_mean_test,y_test)) #

In [ ]:
model.summary()

In [ ]:
model.predict(z_mean)

In [ ]:
model.summary()